!pip install requests

## Log-in Everytime website

In [1]:
import requests

USER_ID = 'cyh0967'
USER_PASSWORD = 'po1545718'

params = {'userid': USER_ID, 'password': USER_PASSWORD}
login_response = requests.post('https://everytime.kr/user/login', data = params)
cookie = login_response.cookies

## Get access token required for query

In [2]:
token_url = "http://everytime.kr/lecture?campus_id=1&page=%d" % 1
token_response = requests.post(token_url, cookies = cookie)

## Parse HTML to get access token which is hidden

In [3]:
from io import StringIO
import re
user_token = None
raw_input = StringIO(token_response.text)
for line in raw_input.readlines():
    if '<input type="hidden" id="userToken" value' in line:
        p = re.compile('value=".*"')
        user_token = p.search(line.strip()).group()[7:-1]
        print(user_token)

t6FJsfnhID5VkcEohzaI8UhZNOpzF2eDRFTLqv59mOzDgamql8S9BgpulpC0nx/xIVwVBHY3eNBYx0rpWhjQE07cCGRbO8XmVx1SLJdy+Pf6/O6JI+6THLLAVf5LXxYr


## Get lecture comments from everytime server

In [4]:
params = {
    'campus_id': '1',
    'limit_num': '100000',
    'start_num': '0',
    'token': user_token,
}

timetable_server_url = "http://timetable.everytime.kr/ajax/lecture/getAllArticles"
comment_response = requests.post(timetable_server_url, data = params, cookies = cookie)

## Collect lecture IDs

In [5]:
import xmltodict

comment_dict = xmltodict.parse(comment_response.text)

In [6]:
lecture_id = []

for comment in comment_dict['response']['article']:
    lecture_id.append(comment['@lecture_id'])

## Get lecture informations

!pip install tqdm

In [8]:
import time
from tqdm import tqdm

lecture_data = {
    'name':[], 'professor':[], 'rate':[],
    'grade':[], 'homework':[], 'team':[], 'attendance':[], 'exam_times':[]
}

for i in tqdm(range(len(lecture_id))):
    params = {
        'school_id': '1',
        'limit_num': '1',
        'token': user_token,
        'lecture_id': lecture_id[i]
    }

    timetable_server_url = "http://timetable.everytime.kr/ajax/lecture/getArticles"
    lecture_response = requests.post(timetable_server_url, data = params, cookies = cookie)
    
    lecture_dict = xmltodict.parse(lecture_response.text)
    
    lecture_data['name'].append(lecture_dict['response']['lecture']['@name'])
    lecture_data['professor'].append(lecture_dict['response']['lecture']['@professor'])
    lecture_data['rate'].append(lecture_dict['response']['rate'])
    
    lecture_data['grade'].append(lecture_dict['response']['details']['@assessment_grade'])
    lecture_data['homework'].append(lecture_dict['response']['details']['@assessment_homework'])
    lecture_data['team'].append(lecture_dict['response']['details']['@assessment_team'])
    lecture_data['attendance'].append(lecture_dict['response']['details']['@assessment_attendance'])
    lecture_data['exam_times'].append(lecture_dict['response']['details']['@exam_times'])
    time.sleep(0.05)

100%|██████████| 25668/25668 [30:27<00:00, 14.13it/s]


## Make DataFrame with lecture informations

In [12]:
from pandas import DataFrame

lecture_df = DataFrame.from_dict(lecture_data)

In [13]:
lecture_df.loc[:10, 'grade']

0     비율 채워줌
1       학점느님
2       학점느님
3       학점느님
4       학점느님
5       학점느님
6     비율 채워줌
7       학점느님
8     비율 채워줌
9     비율 채워줌
10    비율 채워줌
Name: grade, dtype: object

## Save DataFrame to csv file

In [15]:
lecture_df.to_csv('everytime_lecture.csv', encoding='utf-8')